# With excel

In [ ]:
import xlwings as xw
import json
import os

# --- CONFIGURATION ---
# In Jupyter, we use the current working directory.
# Adjust this string if your Excel file is in a different subfolder relative to your notebook.
# Assuming structure: /project_root/inputs/excels/
TARGET_EXCEL = r"C:\Users\VDAM4LZ2\PoC\pilatus_poc\inputs\excel\ER_24_000618_01_Upper_Panel_Analysis_Intact.xlsx"

def calculate_with_excel(json_data):
    print(f"\n--- PHASE 2: EXCEL DRIVER ---")
    print(f"Target File: {TARGET_EXCEL}")
    
    # Check if file exists before trying to open Excel
    if not os.path.exists(TARGET_EXCEL):
        print(f"❌ Error: Excel file not found at {TARGET_EXCEL}")
        print("Please check your folder structure or update TARGET_EXCEL path.")
        return None

    results = json_data.get("Results", {})
    output_data = {}

    # Start Excel
    # visible=True is recommended for debugging so you can see if it opens
    app = xw.App(visible=True) 
    
    try:
        wb = app.books.open(TARGET_EXCEL)
        
        # --- SCENARIO: Upper Skin Panel Analysis ---
        if "Upper_Skin_Panel" in results.get("Elements", {}):
            # Attempt to grab the sheet. 
            # Note: Ensure sheet name '8' matches your specific Excel version
            sheet = wb.sheets['8'] 
            
            # Get the loads from Phase 1
            forces = results["Elements"]["Upper_Skin_Panel"]["Forces"]
            
            # We iterate through the load cases extracted
            for lc_id, load in forces.items():
                if isinstance(load, str): continue # Skip error strings
                
                print(f"Processing LC {lc_id} in Excel...")
                
                # 1. WRITE INPUTS 
                # (Overwriting cells B33, C33, D33 based on your CSV analysis)
                sheet.range('B33').value = load.get('Fx_Nmm', 0.0)
                sheet.range('C33').value = load.get('Fy_Nmm', 0.0)
                sheet.range('D33').value = load.get('Fxy_Nmm', 0.0)
                
                # 2. CALCULATE
                # xlwings/Excel handles this instantly upon write
                
                # 3. READ OUTPUTS
                # (Reading RF from cell P33 based on your CSV analysis)
                rf_result = sheet.range('P33').value 
                
                print(f"   -> Inputs: Fx={load.get('Fx_Nmm'):.3f}, Fy={load.get('Fy_Nmm'):.3f}")
                print(f"   -> Excel Calculated RF: {rf_result}")
                
                # Store result
                status = "PASS"
                if rf_result is None:
                    status = "ERROR"
                elif isinstance(rf_result, (int, float)) and rf_result > 1.0:
                    status = "PASS"
                else:
                    status = "FAIL"

                output_data[lc_id] = {
                    "Method": "Excel_Native",
                    "RF": rf_result,
                    "Status": status
                }

    except Exception as e:
        print(f"❌ Excel Error: {e}")
        return None
    finally:
        # Clean up - Close workbook without saving to preserve template
        try:
            wb.close()
            app.quit()
        except:
            pass
        
    return output_data

# --- RUN CHECK ---
# Load the data generated in Phase 1
INPUT_JSON = "phase1_extracted_data.json"

try:
    with open(INPUT_JSON, "r") as f:
        data = json.load(f)
        
    print("✅ JSON loaded. Launching Excel...")
    excel_results = calculate_with_excel(data)
    
    print("\n--- FINAL RESULTS ---")
    print(json.dumps(excel_results, indent=4))
    
except FileNotFoundError:
    print(f"❌ Could not find {INPUT_JSON}. Please run the Phase 1 script first.")

# Without Excel

In [2]:
import json
import math
import pandas as pd

# --- CONFIGURATION ---
INPUT_FILE = "phase1_extracted_data.json"
OUTPUT_FILE = "phase2_results.json"

# --- ENGINEERING ALLOWABLES (Derived from your CSV uploads) ---
# In a full prod system, these would be read dynamically from the Excel sheets.
ALLOWABLES = {
    "Upper_Skin_Panel": {
        "Shear_Allowable": 28.56,  # N/mm (from 'Stress(Elastic)' in your CSV)
        "Compression_Allowable": 26.03 # N/mm
    },
    "Front_Spar_Splice": {
        "Joint_Shear_Allowable": 1334.0, # N (from 'Ps all' in your CSV)
        "Bearing_Allowable": 2500.0 # N (Estimated for demo)
    }
}

def calculate_panel_margins(element_name, loads):
    """
    Mimics 'ER_24_000618...Upper_Panel_Analysis.xlsx'
    Logic: RF = Allowable / Applied Load
    """
    analysis_results = {}
    
    for lc, forces in loads.items():
        if isinstance(forces, str): continue # Skip error messages
        
        # 1. Get Applied Forces (Absolute values)
        fx = abs(forces["Fx_Nmm"])
        fxy = abs(forces["Fxy_Nmm"])
        
        # 2. Get Allowables
        allow_comp = ALLOWABLES["Upper_Skin_Panel"]["Compression_Allowable"]
        allow_shear = ALLOWABLES["Upper_Skin_Panel"]["Shear_Allowable"]
        
        # 3. Calculate RFs
        # Avoid division by zero
        rf_comp = allow_comp / fx if fx > 0.001 else 99.99
        rf_shear = allow_shear / fxy if fxy > 0.001 else 99.99
        
        # 4. Determine Critical Mode
        if rf_comp < rf_shear:
            critical_rf = rf_comp
            mode = "Compression"
        else:
            critical_rf = rf_shear
            mode = "Shear"
            
        analysis_results[lc] = {
            "Applied_Load": max(fx, fxy),
            "Allowable": allow_comp if mode == "Compression" else allow_shear,
            "RF": round(critical_rf, 2),
            "Failure_Mode": mode
        }
    return analysis_results

def calculate_joint_margins(freebody_name, loads):
    """
    Mimics 'ER_24_000618...Joint_Analysis.xlsx'
    Logic: Calculate Resultant Load -> Compare to Allowable
    """
    analysis_results = {}
    
    for lc, forces in loads.items():
        if isinstance(forces, str): continue
        
        # 1. Calculate Resultant Force (Vector Sum)
        fx = forces["Fx"]
        fy = forces["Fy"]
        fz = forces["Fz"]
        resultant_force = math.sqrt(fx**2 + fy**2 + fz**2)
        
        # 2. Get Allowable
        allowable = ALLOWABLES["Front_Spar_Splice"]["Joint_Shear_Allowable"]
        
        # 3. Calculate RF
        rf = allowable / resultant_force if resultant_force > 0 else 99.99
        
        analysis_results[lc] = {
            "Applied_Load": round(resultant_force, 2),
            "Allowable": allowable,
            "RF": round(rf, 2),
            "Failure_Mode": "Resultant Shear"
        }
    return analysis_results

# --- MAIN EXECUTION ---
def main():
    # 1. Load Phase 1 Data
    try:
        with open(INPUT_FILE, "r") as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"❌ Error: {INPUT_FILE} not found. Run Phase 1 script first.")
        return

    final_report_data = {}

    # 2. Process Elements (Panels)
    print("--- Processing Panel Margins ---")
    if "Elements" in data["Results"]:
        for name, info in data["Results"]["Elements"].items():
            print(f"Analyzing {name}...")
            margins = calculate_panel_margins(name, info["Forces"])
            final_report_data[name] = margins

    # 3. Process Freebodies (Joints)
    print("\n--- Processing Joint Margins ---")
    if "Freebodies" in data["Results"]:
        for name, info in data["Results"]["Freebodies"].items():
            print(f"Analyzing {name}...")
            margins = calculate_joint_margins(name, info["Loads"])
            final_report_data[name] = margins

    # 4. Save Phase 2 Results
    with open(OUTPUT_FILE, "w") as f:
        json.dump(final_report_data, f, indent=4)
        
    print(f"\n✅ PHASE 2 COMPLETE. Results saved to {OUTPUT_FILE}")
    
    # 5. Peak at the critical result for the User
    # Find minimum RF across all checks
    min_rf = 100.0
    critical_component = ""
    for comp, results in final_report_data.items():
        for lc, res in results.items():
            if res["RF"] < min_rf:
                min_rf = res["RF"]
                critical_component = f"{comp} (LC {lc})"
    
    print(f"⚠️  CRITICAL FINDING: {critical_component} has RF = {min_rf}")

if __name__ == "__main__":
    main()

--- Processing Panel Margins ---
Analyzing Upper_Skin_Panel...

--- Processing Joint Margins ---
Analyzing Front_Spar_Splice...

✅ PHASE 2 COMPLETE. Results saved to phase2_results.json
⚠️  CRITICAL FINDING: Front_Spar_Splice (LC 3) has RF = 0.11
